# Chapter 5: The life-cycle model and intertemporal choice

Fehr and Kindermann (2018), Introduction to Computational Economics Using Fortran, Oxford University Press.

https://github.com/fabiankindermann/ce-fortran

In [1]:
# Import package
import pandas as pd
import numpy as np
from numpy.linalg import inv
from scipy import optimize
from scipy.linalg import cholesky
from scipy import stats
from prettytable import PrettyTable

## 5.1 Why do people save?

## 5.1.1 Optimal Savings in a certain world

### 5.1m Module globals

In [3]:
_doc_="""
This module defines the utility function to determine optimal savings in a certain world. 


"""
#Parameters
w =1.0                  # constant wage for all 3 periods
R = 1.0                 # constant return on saving asset
beta = 1.0              # beta is the time discount factor  
gamma = 0.5             # intertemporal elasticity of substitution
egam = 1.0 - 1.0/gamma  # exponent gamma (1/gamma -> relative risk aversion)

#Defining Shapes
a = np.zeros(3)
c = np.zeros(3)

#Define the utility function
def utility(x):
    """ Calculates the negative utility in a certain world. 
    This allows the utility to be maximized through minimzation. 
    The household is subject to budget constraints in each period (see consumption).
    
    Args:
        x(tuple, length 2): saving asset in period 2 and 3 (in period 1 = 0) 
        
    Global Variables:
        a(array of float 64, length 3): saving asset
        c(array of float 64, length 3): consumption 
    
    Returns:
        float64: Utility 
    """
    global c , a 
    
    #savings:
    a[0] = 0        #saving asset period 1
    a[1] = x[0]     #saving asset period 2
    a[2] = x[1]     #saving asset period 3 
    
    #consumption using budget constraints (insure consumption > 0):
    c[0] = w - a[1]             #consumption of period 1
    c[1] = R*a[1] + w - a[2]    #consumption of period 2
    c[2] = R*a[2]               #consumption of period 3
    c    = np.maximum(c,1e-10)  #insures that consumptions is nonnegative to avoid errors
    
    #utility function:
    utility = -(c[0]**egam + beta*c[1]**egam + beta**2*c[2]**egam)/egam #negative sign for maximization
    
    return utility

### 5.1 Program Household 1

In [4]:
""" 
This program solves the constrained maximization problem for the utility function defined in 5.1m.
It calculates the optimal savings in a certain world.
It uses the the function "optimize minimze" from scipy using Sequential Least Squared Programming (SLSQP).
"""

#Defining Shapes
low = np.zeros(2)
up = np.zeros(2)
x = np.zeros(2)

#lower and upper border and initial guess
up = (w,R*w + w)                            # Assets cannot exceed resources in period 2 and 3
bounds = [(low[i],up[i]) for i in range(2)] # Lower bound is 0 -> no debt possible
x = (w/2,(R*w + w)/2)                       # Initial guess

# minimization routine
result = optimize.minimize(utility,x,       # Minimizes the utility function by changing the saving asset
                  method='SLSQP',           # Defines method for minimization
                  bounds=bounds,            # Introduces constraints  
                  options={'disp':True})



#output
t = PrettyTable(['AGE', 'CONS', 'WAGE', 'INC', 'SAV'])
t.add_row([1,round(c[0],2), round(w,2), round(w,2), round(a[1],2)])
t.add_row([2,round(c[1],2), round(w,2), round((w+R*a[1]),2), round(a[2],2)])
t.add_row([3,round(c[2],2), round(0.0,2), round((R*a[2]),2), 0.0])
print(t)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.500000010443049
            Iterations: 6
            Function evaluations: 20
            Gradient evaluations: 6
+-----+------+------+------+------+
| AGE | CONS | WAGE | INC  | SAV  |
+-----+------+------+------+------+
|  1  | 0.67 | 1.0  | 1.0  | 0.33 |
|  2  | 0.67 | 1.0  | 1.33 | 0.67 |
|  3  | 0.67 | 0.0  | 0.67 | 0.0  |
+-----+------+------+------+------+


## 5.1.2 Uncertain Labour Income and Precautionary Savings


### Subroutines from toolbox: normal discrete

In [2]:
def normal_discrete_1(n,mu=0.0,sigma=1.0):
    """
    This function discretizes a normal distribution using the Gauss-Hermite quadrature method.
    It generates n quadrature nodes x and weights prob.
    
    Args:
       n(integer): number of nodes
       mu(float): mean of normal distribution; default 0.0
       sigma(float): variance of normal distribution; default 1.0
    
    Raises:
        Error: normal_discrete sigma has negative value
        Error: Could not discretize normal distribution
       
    Returns:
        x(array, length n): node values
        prob(array, length n): weights of node
    """
    maxit = 200              #set maximum number of iterations
    mu_c = mu                #rename input
    sigma_c = np.sqrt(sigma) #calculate standard deviation
    
    #Insure that sigma >0
    if sigma_c < 0.0:
        print('error:','normal_discrete:','sigma has negative value')
        

    #calculate 1/pi^0.25
    pim4 = 1.0/np.pi**0.25
    
    #get number of points
    m = int((n+1)/2) 
    
    #initialize x and prob
    prob = np.zeros(n)
    x = np.zeros(n) 
    
    
    z1 = 0.0
    
    #Approximates the value of integrals
    #start iteration    
    for i in range(m):
        if i == 0:
            z = np.sqrt(float(2*n+1))- 1.85575 * (float(2*n+1)**(-1.0/6.0))
        elif i == 1:
            z = z - 1.14*(float(n)**0.426)/z
        elif i == 2:
            z = 1.86*z + 0.86*x[0]
        elif i == 3:
            z = 1.91*z+0.91*x[1]
        else:
            z = 2.0*z+x[i-2]
            
        #root finding iterations
        its = 0
        while its<maxit:
            its = its+1
            p1  = pim4
            p2  = 0.0

            for j in range(1,n+1):
                p3 = p2
                p2 = p1
                p1 = z * np.sqrt(2.0/float(j))*p2-np.sqrt(float(j-1)/float(j))*p3

            pp = np.sqrt(2.0*float(n))*p2
            z1 = z
            z  = z1-p1/pp
            if abs(z-z1) < 1e-14:
                break

        if its >= maxit:
            print('Could not discretize normal distribution')

        x[n-1-i] = z 
        x[i] = -z
        prob[i] = 2.0/pp**2
        prob[n-1-i] = prob[i] 
    
    #set up data
    prob = prob/np.sqrt(np.pi)         #calculates weight
    x = x*np.sqrt(2.0)*sigma_c + mu_c  #calculates value of node
    
    return x , prob

### Subroutines from toolbox: log_normal Discrete_1 

In [3]:
def log_normal_discrete_1(x, mu=np.exp(0.5), sigma=np.exp(1)*(np.exp(1)-1)):
    """
    This function discretizes a log-normal distribution using the Gauss-Hermite quadrature method.
    It uses the function normal_discrete_1.
    
    Args:
        x(integer): number of nodes
        mu(float): mean of log-normal distribution; default e^0.5
        sigma(float): variance of normal distribution; default e^1 * (e^1 - 1)
        
    Raises:
        Error: log_normal_discrete sigma has negative value
        Error: log_normal_discrete mu has zero or negative value
    
    Returns:
        x(array, length n): node value
        probt(array, length n): weight of node
    
    """
    #Initialize variables
    mu_c = mu        #rename variable
    sigma_c = sigma  #rename variable
    
    #ensure that sigma is positive
    if sigma_c < 0:
        print('error: log_normal_discrete','sigma has negative value')
    
    #ensure that mu is >0
    if mu_c < 0:
        print('error: log_normal_discrete','mu has zero or negative value')
        
    #get expectation and variance
    sigma_c = np.log(1+sigma_c/mu_c**2) #convert to normal
    mu_c = np.log(mu_c)-0.5*sigma_c     #convert to normal
    
    #check for right array sizes
    #assert np.size(x) ==1 and np.size(prob) ==1, 'normal discrete'
    
    result = normal_discrete_1(x, mu_c, sigma_c) #discretize the normal distribution
    x = result[0]       #extract normal nodes
    x = np.exp(x)       #convert normal nodes back to log-normal
    prob = result[1]    #extract weights
    
    return x, prob

### 5.2m Module globals

In [7]:
_doc_="""
This module defines the utility function for a world with uncertain labour income and precautionary saving.
It also defines the functions to calculate expected values and standard deviation with respect to this 
uncertainty.
Assume wages in period 2 to be log-normally distributed with mean mu_w and variance sig_w.
"""

#Parameters
mu_w = 1.0   # Mean of wage
sig_w = 0.8  # Variance of wage 
n_w = 5      # Number of nodes

R = 1.0                  # Constant return on saving asset
beta = 1.0               # beta is the time discount factor
gamma = 0.5              # intertemporal elasticity of substitution
egam = 1.0 - 1.0/gamma   # exponent gamma (1/gamma -> relative risk aversion)

#Defining Shapes
w = np.zeros(n_w)
weight_w = np.zeros(n_w)
a = np.zeros([3,n_w])
c = np.zeros([3,n_w])
wag = np.zeros([3,n_w])
inc = np.zeros([3,n_w])
sav = np.zeros([3,n_w])

#Define the utility function
def utility(x):
    """Calculates the negative utility at each node when labour income is uncertain.
    The negtative sign allows the utility to be maximized through minimzation. 
    The household is subject to budget constraints in each period (see consumption).
    
    Saving asset in period 1 is 0. 
    Saving asset in period 2 is determined by the saving decision in period 1.
    Saving asset in period 3 is calculated for each n_w realization of wage in period 2.  
    
    Consumption in period 1 depends on the wage and savings decision. 
    Consumption in period 2-3 depends on the realization of wage and savings in period 2.
    
    Utility in period 1 depends on the saving decision. Utility in period 2-3 is calculated for each 
    realization of wage in period 2. 
    
    Args:
        x(array of float 64, size (1+n_w)): saving asset in period 2 and 3 (in period 1 = 0) 
        
    Global Variables:
        a(array of float 64, size (3,n_w)): saving asset for period 1-3 at each node 
        c(array of float 64, size (3,n_w)): consumption for period 1-3 each node
    
    Returns:
        array of float64, size (1+n_w): Utility
    
    """
    global c,a 
    
    #savings
    a[0,:] = 0.0          #saving asset period 1
    a[1,:] = x[0]         #saving asset period 2
    a[2,:] = x[1:(1+n_w)] #saving asset period 3; introduce uncertainty of wage
        
    #consumption (ensure consumption > 0)
    c[0,:] = mu_w - a[1,0]             #consumption period 1
    c[1,:] = R*a[1,0] + w[:] - a[2,:]  #consumption period 2
    c[2,:] = R*a[2,:]                  #consumption period 3
    c = np.maximum(c,1e-10)            #ensure that consumption is nonnegative to avoid errors
    
    #expected utility of periods 2 and 3
    utility = 0.0
    for iw in range (n_w):  #calculates utility at each node 
        utility = utility + weight_w[iw]*(c[1,iw]**egam + beta*c[2,iw]**egam)
    #add the utility of period 1
    utility = -(c[0,0]**egam + beta*utility)/egam #negative sign for maximization
    
    return utility
    

#Calculate the Expected value of x
def E(x):
    """Calculates the expected value when labour income is uncertain"""
    #calculate expected value
    E = 0.0 
    for iw in range (n_w):
        E = E + x[iw]*weight_w[iw]
    
    return E

#Calculates the Standard deviation of x
def Std(x):
    """Calculates the standard deviation when labour income is uncertain"""
    #calculates expected value
    E = 0.0
    for iw in range(n_w):
        E = E + x[iw]*weight_w[iw]
    
    #calculate standard deviation
    Std = 0.0
    for iw in range(n_w):
        Std = Std + x[iw]**2*weight_w[iw]
    Std = np.sqrt(np.maximum(Std-E**2,0.0))
    
    return Std


### Program Household 2

In [21]:
""" This program solves the constrained maximization problem for the utility function defined in 5.2m.
It assumes uncertainty of income in the second period i.e. wages to be log-normally distributed in period 2.
First, the distribution of wages from period 2 is discretized generating n_w quadrature nodes w 
and weights weight_w.
Then the program minimizes the negative utility function of a household with uncertain labour income.
For this it uses n_w savings level a3 for any of the nodes w as well as a2 as input.
The program uses the the function "optimize minimze" from scipy with method SLSQP to find the optimal saving.
We obtain consumption and the saving asset both depending on the realization of wage.
Finally, we compute the mean and standard deviations of consumption, income and savings for the 3 periods.
"""

#discretize w
#Generates n_w quadrature nodes w and weights weight_w for period 2
w = (log_normal_discrete_1(n_w, mu_w, sig_w))[0]        #discretized random variable wage
weight_w = (log_normal_discrete_1(n_w, mu_w, sig_w))[1] #weight for wage

#Lower and upper border and initial guess
low = np.zeros(1+n_w)                            #lower bound = 0 -> no debt possible
up= np.zeros(n_w+1)                              #shape of size 6 of upper bound (a2,a3.1,a3.2,a3.3,a3.4,a3.5)
up[0]= mu_w                                      #a2
up[1:n_w+1] = R*mu_w+w                           #upper level for realization of w (a3.1,a3.2,a3.3,a3.4,a3.5)
bounds = [(low[i],up[i]) for i in range(1+n_w)]  #Combine the bounds
x = up/2.0                                       #Initial guess 

#minimization routine
result = optimize.minimize(utility,x,    #Minimizes the utility function by changing the saving asset
                  method='SLSQP',        #Define method for minimization
                  bounds=bounds,         #Introduce constraints
                  options={'disp':True})



#set up data for output
for iw in range (n_w):
    wag[0,iw] = mu_w   #wage period 1
    inc[0,iw] = mu_w   #income period 1
    sav[0,iw] = a[1,0] #saving period 1 (i.e. saving asset of period 2)
    
    wag[1,iw] = w[iw]            #wage period 2
    inc[1,iw] = w[iw] + R*a[1,0] #income period 2 
    sav[1,iw] = a[2,iw]          #saving period 2 (i.e. saving asset of period 3)
    
    wag[2,iw] = 0.0      #wage period 3
    inc[2,iw] = R*a[2,0] #income period 3
    sav[2,iw] = 0.0      #saving period 3 (0 because we consume everything in the last period)
  


#output
#calculate mean and standard deviation
t = PrettyTable(['AGE', 'CONS', 'WAGE', 'INC', 'SAV'])
t.add_row([1,round(E(c[0,:]),2), round(E(wag[0,:]),2), round(E(inc[0,:]),2), round(E(sav[0,:]),2)])
t.add_row(['Std' ,round(Std(c[0,:]),2), round(Std(wag[0,:]),2), round(Std(inc[0,:]),2), round(Std(sav[0,:]),2)])
t.add_row([2,round(E(c[1,:]),2), round(E(wag[1,:]),2), round(E(inc[1,:]),2), round(E(sav[1,:]),2)])
t.add_row(['Std' ,round(Std(c[1,:]),2), round(Std(wag[1,:]),2), round(Std(inc[1,:]),2), round(Std(sav[1,:]),2)])
t.add_row([3,round(E(c[2,:]),2), round(E(wag[2,:]),2), round(E(inc[2,:]),2), round(E(sav[2,:]),2)])
t.add_row(['Std' ,round(Std(c[2,:]),2), round(Std(wag[2,:]),2), round(Std(inc[2,:]),2), round(Std(sav[2,:]),2)])
print(t)

E_w = sum(weight_w*w)
Var_w = sum(weight_w*w**2) - sum(weight_w*w)**2

print(" E(w)=",round(E_w,1))
print(" Var(w)=",round(Var_w,1))



Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.284512341140482
            Iterations: 14
            Function evaluations: 103
            Gradient evaluations: 14
+-----+------+------+------+------+
| AGE | CONS | WAGE | INC  | SAV  |
+-----+------+------+------+------+
|  1  | 0.54 | 1.0  | 1.0  | 0.46 |
| Std | 0.0  | 0.0  | 0.0  | 0.0  |
|  2  | 0.73 | 1.0  | 1.46 | 0.73 |
| Std | 0.43 | 0.89 | 0.89 | 0.47 |
|  3  | 0.73 | 0.0  | 0.27 | 0.0  |
| Std | 0.47 | 0.0  | 0.0  | 0.0  |
+-----+------+------+------+------+
 E(w)= 1.0
 Var(w)= 0.8


## 5.1.3 Uncertain Capital and Labour Income


### 5.3m Module globals

In [22]:
_doc_="""
This module defines the utility function for a world with uncertain capital and labour income.
It also defines the functions to calculate expected values and standard deviation with respect to this 
uncertainty.
Assume wages  in period 2 to be log-normally distributed with mean mu_w and variance sig_w.
Assume return on capital to be log-normally distributed with mean mu_R and variance sig_R
and to be independent over time.

"""

#Parameters
mu_w = 1.0   #mean of wage
sig_w = 0.4  #variance of wage
n_w = 5      #number of wage nodes

mu_R = 1.0   #mean of return on saving asset
sig_R = 0.4  #varianges of return on saving asset
n_R = 5      #number of return nodes

beta = 1.0             #beta is the time discount factor
gamma = 0.5            #intertemporal elasticity of substitution
egam = 1.0 - 1.0/gamma #exponent gamma (1/gamma -> relative risk aversion)

#Defining Shapes
w = np.zeros(n_w)
weight_w = np.zeros(n_w)
R = np.zeros(n_R)
weight_R = np.zeros(n_R)
a = np.zeros(shape=(3,n_w,n_R))
c = np.zeros(shape=(3,n_w,n_R, n_R))
wag = np.zeros(shape=(3,n_w,n_R, n_R))
inc = np.zeros(shape=(3,n_w,n_R, n_R))
sav = np.zeros(shape=(3,n_w,n_R, n_R))


#utility function of the household
def utility(x):
    """Calculates the negative utility at each node when capital and labour income is uncertain.
    The negative sign allows the utility to be maximized through minimzation. 
    The household is subject to budget constraints in each period (see consumption).
    
    Saving asset in period 1 is 0.
    Saving asset in period 2 is determined by the saving decision in period 1.
    Saving asset in period 3 is calculated for each n_w realization of wage and each n_R 
    realization of return on capital.
    
    Consumption in period 1 depends on the wage and savings decision. 
    Consumption in period 2-3 is calculated for all n_w and n_R realizations of wage and return on capital.
    
    Utility in period 1 depends on the saving decision.  
    Utility in period 3 is calculated for all possible consumptions.
    
    Args:
        x(array of float 64, size (1+n_w*n_R)): saving asset in period 2 and 3 (in period 1 = 0) 
        
    Global Variables:
        a(array of float 64, size (3,5,5)): saving asset for period 1-3 at each node
        c(array of float 64, size (3,5,5,5)): consumption for periods 1-3 at each node
    
    Returns:
        array of float64, length 26: Utility
    
    """
    global c,a

    prob = 0.0
    #savings
    a[0,:,:] = 0    #saving asset period 1
    a[1,:,:] = x[0] #saving asset period 2
    ic = 1          
    
    for iw in range(n_w):       #for loop for each realization of wage
        for ir2 in range(n_R):  #for loop for each realization of return on capital
            a[2,iw,ir2] = x[ic] #saving asset period 3 for all realizations of wage and return on capital
            ic = ic+1
            
    #consumption
    c[0,:,:,:] = mu_w - a[1,0,0] #consumption in period 1 
    for iw in range(n_w):        #for loop for each realization of wage
        for ir2 in range(n_R):   #for loop for each realization of return on capital in period 2
            c[1,iw,ir2,:] = R[ir2] * a[1,0,0] + w[iw] - a[2,iw,ir2] #consumption in period 2
            for ir3 in range(n_R): #for loop for each realization of return on capital in period 3
                c[2,iw,ir2,ir3] = R[ir3]*a[2,iw,ir2] #consumption in period 3
                                                            
    c = np.maximum(c,1e-10) #ensure that consumption is nonnegative to avoid errors
    
    #expected utility of periods 2 and 3
    utility = 0.0
    for iw in range(n_w):          #for loop for each realization of wage
        for ir2 in range(n_R):     #for loop for each realization of return on capital in period 2
            for ir3 in range(n_R): #for loop for each realization of return on capital in period 3
                prob = float(weight_w[iw]*weight_R[ir2]*weight_R[ir3]) #combined weight 
                utility = utility+prob*(c[1,iw,ir2,0]**egam + beta*c[2,iw,ir2,ir3]**egam)#utility for period 2-3
                
    result = -(c[0,0,0,0]**egam + beta*utility)/egam #add utility for period 1
    return result
    

#calculates the expected value of x    
def E(x):
    """Calculates the expected value when labour and capital income is uncertain"""
    E = 0
    for iw in range(n_w):
        for ir2 in range(n_R):
            for ir3 in range(n_R):
                E = E + x[iw,ir2,ir3]*weight_w[iw]*weight_R[ir2]*weight_R[ir3]
                
    return E
                
#calculates the standard deviation of x
def Std(x):
    """Calculates the standard deviation when labour and capital income is uncertain"""
    #calculate the expected value
    E= 0
    for iw in range(n_w):
        for ir2 in range(n_R):
            for ir3 in range(n_R):
                E = E + x[iw,ir2,ir3]*weight_w[iw]*weight_R[ir2]*weight_R[ir3]
    
    #calculate the standard deviation
    Std = 0
    for iw in range(n_w):
        for ir2 in range(n_R):
            for ir3 in range(n_R):
                Std = Std + x[iw,ir2,ir3]**2*weight_w[iw]*weight_R[ir2]*weight_R[ir3]
            
    Std = np.sqrt(np.maximum(Std-E**2,0))
    
    return Std

### 5.3 Program Household 3

In [23]:
"""This program solves the constrained maximization problem for the utility function defined in 5.3m.
It assumes uncertainty of income in the second period i.e. wages to be log-normally distributed in period 2.
Furthermore, it assumes uncertainty of return on capital i.e. to be log-normally distriributed in period 2-3.
First, the distribution of wages from period 2 is discretized generating n_w quadrature nodes w 
and weights weight_w.
Then the distribution of capital return from period 2 and 3 is discretized generating n_R  quadrature nodes R
and weights weight_R.
Then the program minimizes the negative utility function of a household with uncertain 
labour and capital income.
For this it uses n_w*n_R savings level a3 for any of the nodes w as well as a2 as input.
The program uses the the function "optimize minimze" from scipy with method SLSQP to find the optimal saving.
We obtain consumption and the saving asset both depending on the realization of wage and capital return.
Finally, we compute the mean and standard deviations of consumption, income and savings for the 3 periods.
"""

#discretize w
#Generates n_w quadrature nodes w and weights weight_w for period 2
w = (log_normal_discrete_1(n_w, mu_w, sig_w))[0]        #discretized random variable wage
weight_w = (log_normal_discrete_1(n_w, mu_w, sig_w))[1] #weight for wage

#discretize R
#Generates n_R quadrature nodes R and weights weight_R for period 2
R = (log_normal_discrete_1(n_R, mu_R, sig_R))[0]        #discretized random variable wage
weight_R = (log_normal_discrete_1(n_R, mu_R, sig_R))[1] #weight for wage


#Lower and upper border and initial guess
#low = 0.0
low = np.zeros(1+n_w*n_R) #lower bound = 0 -> no debt possible
up= np.zeros(1+n_w*n_R)   #shape of size 26 for upper bound (a2,a3.1.1,a3.1.2,...,a3.1.n_R,...,a3.n_w.n_R)

up[0]= mu_w                         #a2
ic = 1
for iw in range(n_w):               #for loop for each realization of wage
    for ir2 in range(n_R):          #for loop for each realization of return on capital
        up[ic] = R[ir2]*mu_w+w[iw]  #upper bound for realizations of wage and return on capital a3.n_w.n_R
        ic = ic+1
        

bounds = [(low[i],up[i]) for i in range(1+n_w*n_R)] #Combine the bounds
x = up/2.0                                          #Initial guess

#minimization routine
result = optimize.minimize(utility,x,    #Minimizes the utility function by changing the saving asset
                  method='SLSQP',        #Define method for minimization
                  bounds=bounds,         #Introduce constraints
                  options={'disp':True})





#output
for iw in range(n_w):
    for ir2 in range(n_R):
        for ir3 in range(n_R):
            wag[0,iw,ir2,ir3] = mu_w     #wage period 1
            inc[0,iw,ir2,ir3] = mu_w     #income period 2
            sav[0,iw,ir2,ir3] = a[1,0,0] #saving period 3 (i.e. saving asset of period 2)
            
            wag[1,iw,ir2,ir3] = w[iw]                   #wage period 2
            inc[1,iw,ir2,ir3] = w[iw] + R[ir2]*a[1,0,0] #income period 2
            sav[1,iw,ir2,ir3] = a[2,iw,ir2]             #saving period 2 (i.e. saving asset of period 3)
            
            wag[2,iw,ir2,ir3] = 0                  #wage period 3
            inc[2,iw,ir2,ir3] = R[ir3]*a[2,iw,ir2] #income period 3
            sav[2,iw,ir2,ir3] = 0           #saving period 3(0 because we consume everything in the last period)
            

#output
#calculate mean and standard deviation
t = PrettyTable(['AGE', 'CONS', 'WAGE', 'INC', 'SAV'])
t.add_row([1,round(E(c[0,:,:,:]),2), round(E(wag[0,:,:,:]),2), round(E(inc[0,:,:,:]),2), round(E(sav[0,:,:,:]),2)])
t.add_row(['Std' ,round(Std(c[0,:,:,:]),2), round(Std(wag[0,:,:,:]),2), round(Std(inc[0,:,:,:]),2), round(Std(sav[0,:,:,:]),2)])
t.add_row([2,round(E(c[1,:,:,:]),2), round(E(wag[1,:,:,:]),2), round(E(inc[1,:,:,:]),2), round(E(sav[1,:,:,:]),2)])
t.add_row(['Std' ,round(Std(c[1,:,:,:]),2), round(Std(wag[1,:,:,:]),2), round(Std(inc[1,:,:,:]),2), round(Std(sav[1,:,:,:]),2)])
t.add_row([3,round(E(c[2,:,:,:]),2), round(E(wag[2,:,:,:]),2), round(E(inc[2,:,:,:]),2), round(E(sav[2,:,:,:]),2)])
t.add_row(['Std' ,round(Std(c[2,:,:,:]),2), round(Std(wag[2,:,:,:]),2), round(Std(inc[2,:,:,:]),2), round(Std(sav[2,:,:,:]),2)])
print(t)


E_w = sum(weight_w*w)
Var_w = sum(weight_w*w**2) - sum(weight_w*w)**2

print(" E(w)=",round(E_w,1))
print(" Var(w)=",round(Var_w,1))           

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.8170486403218415
            Iterations: 45
            Function evaluations: 1222
            Gradient evaluations: 45
+-----+------+------+------+------+
| AGE | CONS | WAGE | INC  | SAV  |
+-----+------+------+------+------+
|  1  | 0.56 | 1.0  | 1.0  | 0.44 |
| Std | 0.0  | 0.0  | 0.0  | 0.0  |
|  2  | 0.66 | 1.0  | 1.44 | 0.78 |
| Std | 0.31 | 0.63 | 0.69 | 0.38 |
|  3  | 0.78 | 0.0  | 0.78 | 0.0  |
| Std | 0.66 | 0.0  | 0.66 | 0.0  |
+-----+------+------+------+------+
 E(w)= 1.0
 Var(w)= 0.4


# 5.2 Where do people save and invest?

### Subroutines from toolbox: log_normal_discrete_2

In [4]:
def log_normal_discrete_2(n, mu = np.exp(0.5), sigma = np.exp(1.0)*(np.exp(1.0)-1.0), rho=0.0):
    """
    This function discretizes a two-dimensional log-normal distribution 
    using the Gauss-Hermite quadrature method.
    It uses the function normal_discrete_1.
    The function returns n_w quadrature nodes for w and n_R nodes for R 
    due to the possible correlation between the two. (Joint distribution)
    
    Args:
        n(tuple, size 2): number of nodes for n_w and n_R
        mu(tuple, size 2): mean of log-normal distribution for wage and capital return; default e^0.5
        sigma(tuple, size 2): variance of normal distribution for wage and capital return; 
                              default e^1 * (e^1 - 1)
        rho(float, size 1): correlation between wage and capital return; default 0.0
        
    Raises:
        Error: log_normal_discrete sigma has negative value
        Error: log_normal_discrete mu has zero or negative value
        Error: log_normal_discrete who is outside -1 to 1
        Error: Variance-Covariance matrix is not symmetric
    
    Returns:
        x(array of float 64, size (n_w*n_R,2)): node value
        prob(array of float 64, size (n_w*n_R)): weight of node
    
    """
    #Define Shapes
    mu_c = np.zeros(2)
    sig_c = np.zeros(2)
    sigma_c = np.zeros(shape=(2,2))
    x1 = n[0]
    x2 = n[1]
    prob = np.zeros(shape=((n[0]*n[1])))
    x = np.zeros((n[0]*n[1],2))
    
    #initialize Paramters
    sig_c[:] = sigma #rename variable
    mu_c[:] = mu     #rename variable
    rho_c = rho      #rename variable
    
    #ensure that sigma is positive
    if any(sig_c < 0.0):
        print('Error: log_normal_discrete','sigma has negative value')
    #ensure that mu is >0
    if any(mu_c <= 0.0):
        print('Error: log_normal_discrete','mu has zero or negative value')
    #ensure that rho is within -1 and 1
    if rho_c < -1.0 or rho_c > 1.0:
        print('Error: log_normal_discrete','rho is outside -1 to 1')
        
    
    #get expectations and variance
    sig_c = np.log(1.0+sig_c/mu_c**2) #convert to normal
    mu_c = np.log(mu_c)-0.5*sig_c     #convert to normal
    
    #set up covariance matrix
    sigma_c[0,0] = sig_c[0]
    sigma_c[1,1] = sig_c[1]
    sigma_c[0,1] = np.log(rho_c*np.sqrt(np.exp(sig_c[0])-1.0)*np.sqrt(np.exp(sig_c[1])-1.0)+1.0)
    sigma_c[1,0] = sigma_c[0,1]
    
    #Check whether sigma is symmetric
    if np.any(abs(sigma_c.T-sigma_c) > 1e-20):
        print('Error: Variance-Covariance matrix is not symmetric')

    
    #get standard normal distributed random variables
    x_1 , prob_1 = normal_discrete_1(x1, 0.0, 1.0) #discretize the standard normal distribution for n_w
    x_2 , prob_2 = normal_discrete_1(x2, 0.0, 1.0) #discretize the standard normal distribution for n_R
    
    #get joint distribution
    m = 0
    for k in range(n[1]):                 #for loop for n_R nodes
        for j in range(n[0]):             #for loop for n_w nodes
            prob[m] = prob_1[j]*prob_2[k] #get joint probability
            x[m,0] = x_1[j]               #insert n_w values for wage node
            x[m,1] = x_2[k]               #insert n_R values for capital return node
            m = m+1
            
            
    #decompose var-cov matrix 
    #cholesky decomposition of a hermitian, positive-definite matix into
    #the product of a lower triangular matrix
    if not any(abs(sig_c) <= 1e-100):
        l = cholesky(sigma_c, lower = True)
    else:
        l = np.zeros(shape=(2,2))
        l[0,0] = np.sqrt(sig_c[0])
        l[1,1] = np.sqrt(sig_c[1])
        
        
        
    #calculate distribution
    x = x@(l.T)               #matrix multiplication of node values and lower triangular matrix
    x[:,0] = x[:,0] + mu_c[0] #add mean of normal distribution of wage to node
    x[:,1] = x[:,1] + mu_c[1] #add mean of normal distribution of capital return to node
    x = np.exp(x)             #Convert normal distribution to log-normal
    
    return x, prob

## 5.2.1 Uncertain Capital Income and Portfolio Choice

### 5.2m Module globals

In [26]:
_doc_="""
This module defines the utility function for a world with uncertain capital income and portfolio choice.
It also defines the functions to calculate expected values and standard deviation with respect to this 
uncertainty.
Assume that the investment can be allocated to two assets (a riskless and a risky asset) in period 1 and 2.
Assume the joint distribution of labour income and capital return in period 2 is a two-dimensional log-normal 
distribution.
Assume wages  in period 2 to be log-normally distributed with mean mu_w and variance sig_w.
Assume return on capital in period 2 to be log-normally distributed with mean mu_R and variance sig_R.
Assume distribution of R3 to have the same mean and variance as R2, but to be independent.
"""

#Parameters
mu_w = 1.0  #mean of wage
sig_w = 0.5 #variance of wage
n_w = 5     #number of wage nodes

mu_R = 1.22    #mean of return on capital
sig_R = 0.5    #variance of return on capital
n_R = 5        #number of capital return nodes
rho_wR = -0.5  #correlation between wage and capital return

Rf = 1.0               #return on riskless asset
beta = 1.0             #beta is the time discount factor
gamma = 0.5            #intertemporal elasticity of substituion
egam = 1.0 - 1.0/gamma #exponent gamma (1/gamma -> relative risk aversion)

#Defining Shapes
wR = np.zeros(shape = (n_w*n_R,2))
weight_wR = (n_w*n_R)
R = np.zeros(n_R)
weight_R = np.zeros(n_R)
a = np.zeros(shape=(3,n_w*n_R))
omega = np.zeros(shape=(2,n_w*n_R))
c = np.zeros(shape=(3,n_w*n_R,n_R))


#utility function of the household
def utility(x):
    """Calculates the negative utility at each node when capital income and portfolio choice is uncertain.
    The negative sign allows the utility to be maximized through minimzation. 
    The household is subject to budget constraints in each period (see consumption).
    
    Saving asset in period 1 is 0. Saving asset in period 2 is determined by the saving decision 
    in period 1.
    Saving asset in period 3 depends on uncertain wage and return on capital. It is calculated 
    for each n_w realization of wage and each n_R realization of return on capital.
    
    Share of risky asset in period 1 and 2 depends on the investment decision.
    
    Consumption in period 1 depends on the wage and savings decision. 
    Consumption in period 2 depends additionally on the investment decision from period 1 (risky vs. riskless).
    It is calculated for all realizations of wage and return on capital.
    Consumption in period 3 depends on the investment decision from period 2.
    It is calculated for all possible investment decisions and realizations of return.
    
    Utility in period 1 depends on the saving decision. 
    Utility in period 3 is calculated for all states of consumption.
    
    Args:
        x(array of float 64, size (2*(1+n_w*n_R))): saving asset in period 2 and 3 (in period 1 = 0) and
                                                    share of portfolio invested in risky asset.
        
    Global Variables:
        a(array of float 64, size (3,n_w*n_R): saving asset for period 1-3 at each node
        c(array of float 64, size (3,n_w*n_R,n_R): consumption for periods 1-3 at each node
        omega(array of float 64 ,size (2, n_w*n_R)): share of portfolio invested in risky asset at each node
    
    Returns:
        array of float64, size (2*(1+n_w*n_R)): Utility
    
    """
    global c,a,omega

    #savings
    a[0,:] = 0.0      #saving asset period 1
    a[1,:] = x[0]     #saving asset period 2
    omega[0,:] = x[1] #share of portfolio invested in risky asset in period 1
    ic = 2
    iwR = 0
    for iw in range(n_w):          #for loops to decompose x and  assign to variable  
        for ir2 in range(n_R):     
            a[2,iwR] = x[ic]       #saving asset in period 3 (i.e. saving in period 2)
            omega[1,iwR] = x[ic+1] #share of risky asset in period 2
            ic = ic+2
            iwR = iwR+1
            
    #consumption
    c[0,:,:] = mu_w - a[1,0] #consumption in period 1
    iwR = 0
    for iw in range(n_w): #for loop for portfolio allocation at each wage node     
        for ir2 in range(n_R): 
            c[1,iwR,:] = (Rf+omega[0,0]*(wR[iwR,1]-Rf))*a[1,0] + wR[iwR,0] - a[2,iwR] #consumption in period 2
            for ir3 in range(n_R):
                c[2,iwR,ir3] = (Rf+omega[1,iwR]*(R[ir3]-Rf))*a[2,iwR] #consumption in period 3
            iwR = iwR+1
    
    c = np.maximum(c,1e-10) #ensure that consumption is nonnegative to avoid errors
    
    #expected utility of perios 2 and 3
    utility = 0.0
    iwR = 0
    for iw in range(n_w):          #for loop for each realization of wage
        for ir2 in range(n_R):     #for loop for period 2 investment return
            for ir3 in range(n_R): #for loop for period 3 investment return
                prob = weight_wR[iwR]*weight_R[ir3] #combined weight
                utility = utility+prob*(c[1,iwR,0]**egam+beta*c[2,iwR,ir3]**egam) #utility for period 2-3
            iwR = iwR+1
    
    result = -(c[0,0,0]**egam+beta*utility)/egam #add utility in period 1
    
    return result
            
#calculates expected value of x            
def E(x): 
    """Calculates the expected value when capital income and portfolio choice is uncertain"""
    E = 0.0
    ic = 0
    for iw in range(n_w):
        for ir2 in range(n_R):
            for ir3 in range(n_R):
                E = E + x[ic,ir3]*weight_wR[ic]*weight_R[ir3]
            ic = ic+1
    return E
            
#calculates the standard deviation of x            
def Std(x):
    """Calculates the standard deviation when capital income and portfolio choice is uncertain"""
    #calculates expected value
    E = 0.0
    ic = 0
    for iw in range(n_w):
        for ir2 in range(n_R):
            for ir3 in range(n_R):
                E = E + x[ic,ir3]*weight_wR[ic]*weight_R[ir3]
            ic = ic+1
            
    #calculate standard deviation
    Std = 0.0
    ic = 0
    for iw in range(n_w):
        for ir2 in range(n_R):
            for ir3 in range(n_R):
                Std = Std + x[ic,ir3]**2*weight_wR[ic]*weight_R[ir3]
            ic = ic+1
    Std = np.sqrt(np.maximum(Std-E**2,0.0))
    
    return Std

### 5.4 Program Household 4

In [28]:
"""This program solves the constrained maximization problem for the utility function defined in 5.4m.
The household has to decide on the optimal saving as well as the optimal allocation 
to the risky asset in period 1 and 2.
First, the joint distribution of wages and capital returns from period 2 is discretized
generating n_w*n_R quadrature nodes wR and weights weight_wR.
Then the distribution of capital return from period 3 is discretized generating n_R  quadrature nodes R
and weights weight_R.
Then the program minimizes the negative utility function of a household with uncertain 
labour and capital income.
The program uses the the function "optimize minimze" from scipy with method SLSQP to find the 
optimal saving and optimal allocation.
We obtain consumption, the saving asset and the allocation to the risky asset.
Finally, we compute the mean and standard deviations of consumption, income and savings for the 3 periods.
"""
#Define Shapes  
x = np.zeros(shape=(2*(1+n_w*n_R)))
low = np.zeros(shape=(2*(1+n_w*n_R)))
up = np.zeros(shape=(2*(1+n_w*n_R)))
wag = np.zeros(shape=(3,n_w*n_R,n_R))
inc = np.zeros(shape=(3,n_w*n_R,n_R))
sav = np.zeros(shape=(3,n_w*n_R,n_R))
alp = np.zeros(shape=(3,n_w*n_R,n_R))
E_st = np.zeros(2)
Var_st = np.zeros(2)
rho_st = 0

#discretize w and R in period 2
#due to the possible correlation we get n_w * n_R nodes
wR= log_normal_discrete_2((n_w, n_R), (mu_w, mu_R), (sig_w, sig_R), rho_wR)[0] #value of wage & R nodes
weight_wR = log_normal_discrete_2((n_w, n_R), (mu_w, mu_R), (sig_w, sig_R), rho_wR)[1] #weight of wage & R nodes            

#discretize log(R) in period 3    
R= log_normal_discrete_1(n_R, mu_R, sig_R)[0]         #value of R nodes
weight_R = log_normal_discrete_1(n_R, mu_R, sig_R)[1] #weight of R nodes  

#lower and upper border and initial guess
up[0] = mu_w #lower bound for optimal saving 
up[1] = 1.0  #lower bound for omega (i.e. optimal allocation)
ic = 2
iwR = 0

#for loop to set upper bounds for portfolio allocation omega and savings s 
#lower bound for omega = 0 -> no short selling
for iw in range(n_w):
    for ir2 in range(n_R):
        up[ic] = wR[iwR,1]*mu_w+wR[iwR,0] #upper bound for savings 
        up[ic+1] = 1.0                    #upper bound for portfolio allocation
        ic = ic+2
        iwR = iwR+1
        
bounds = [(low[i],up[i]) for i in range(2*(1+n_w*n_R))] #combine lower and upper bounds
x = up/2.0                                            #initial guess

#minimization routine
result = optimize.minimize(utility,x, #minimize the utility function by changing savings and portfolio allocation
                  method='SLSQP',     #Define method for minimization
                  bounds=bounds,      #Introduce constraints
                  tol = 1e-14,        #adjust tolerance level for increase in choice variables
                  options={'disp':True})
  

#set up data for output
iwR = 0
for iw in range(n_w):
    for ir2 in range(n_R):
        for ir3 in range(n_R):
            wag[0,iwR,ir3] = mu_w           #wage period 1
            inc[0,iwR,ir3] = mu_w           #income period 1
            sav[0,iwR,ir3] = a[1,0]         #savings period 1 (i.e. saving asset of period 2)
            alp[0,iwR,ir3] = omega[0,0]*100 #share of investment in risky asset period 1 
            
            wag[1,iwR,ir3] = wR[iwR,0]                                       #wage period 2
            inc[1,iwR,ir3] = wR[iwR,0]+(Rf+omega[0,0]*(wR[iwR,1]-Rf))*a[1,0] #income period 2
            sav[1,iwR,ir3] = a[2,iwR]                                        #saving period 2
            alp[1,iwR,ir3] = omega[1,iwR]*100    #share of investment in risky asset period 2
            
            wag[2,iwR,ir3] = 0.0 #wage period 3
            inc[2,iwR,ir3] = (Rf+omega[1,iwR]*(R[ir3]-Rf))*a[2,iwR] #income period 3
            sav[2,iwR,ir3] = 0.0 #saving period 3
            alp[2,iwR,ir3] = 0.0 #share of investment in risky asset period 3
        iwR = iwR+1
        
 
#output
#calculate mean and standard deviation
t = PrettyTable(['AGE', 'CONS', 'WAGE', 'INC', 'SAV', 'SHARE'])
t.add_row([1,round(E(c[0,:,:]),2), round(E(wag[0,:,:]),2), round(E(inc[0,:,:]),2), round(E(sav[0,:,:]),2), round(E(alp[0,:,:]),2)])
t.add_row(['Std' ,round(Std(c[0,:,:]),2), round(Std(wag[0,:,:]),2), round(Std(inc[0,:,:]),2), round(Std(sav[0,:,:]),2), round(Std(alp[0,:,:]),2)])
t.add_row([2,round(E(c[1,:,:]),2), round(E(wag[1,:,:]),2), round(E(inc[1,:,:]),2), round(E(sav[1,:,:]),2), round(E(alp[1,:,:]),2)])
t.add_row(['Std' ,round(Std(c[1,:,:]),2), round(Std(wag[1,:,:]),2), round(Std(inc[1,:,:]),2), round(Std(sav[1,:,:]),2), round(Std(alp[1,:,:]),2)])
t.add_row([3,round(E(c[2,:,:]),2), round(E(wag[2,:,:]),2), round(E(inc[2,:,:]),2), round(E(sav[2,:,:]),2), round(E(alp[2,:,:]),2)])
t.add_row(['Std' ,round(Std(c[2,:,:]),2), round(Std(wag[2,:,:]),2), round(Std(inc[2,:,:]),2), round(Std(sav[2,:,:]),2), round(Std(alp[2,:,:]),2)])
print(t)

t = PrettyTable(['','w', 'R2', 'R3'])
t.add_row(['E', round(E_st[0],2), round(E_st[1],2), round(sum(weight_R*R),2)])
t.add_row(['Var', round(Var_st[0],2), round(Var_st[1],2), round(sum(weight_R*R**2)-sum(weight_R*R)**2,2)])
t.add_row(['rho(w, R2)', '', round(rho_st,2), ''])
print(t)

 

Iteration limit reached    (Exit mode 9)
            Current function value: 8.225803457947357
            Iterations: 100
            Function evaluations: 5325
            Gradient evaluations: 100
+-----+------+------+------+------+-------+
| AGE | CONS | WAGE | INC  | SAV  | SHARE |
+-----+------+------+------+------+-------+
|  1  | 0.59 | 1.0  | 1.0  | 0.41 | 100.0 |
| Std | 0.0  | 0.0  | 0.0  | 0.0  |  0.0  |
|  2  | 0.76 | 1.0  | 1.5  | 0.74 | 33.51 |
| Std | 0.31 | 0.71 | 0.62 | 0.31 |  1.01 |
|  3  | 0.8  | 0.0  | 0.8  | 0.0  |  0.0  |
| Std | 0.38 | 0.0  | 0.38 | 0.0  |  0.0  |
+-----+------+------+------+------+-------+
+------------+-----+-----+------+
|            |  w  |  R2 |  R3  |
+------------+-----+-----+------+
|     E      | 0.0 | 0.0 | 1.22 |
|    Var     | 0.0 | 0.0 | 0.5  |
| rho(w, R2) |     |  0  |      |
+------------+-----+-----+------+


## 5.2.2 Uncetain Lifespan and annuity choice

### 5.5 Module globals

In [7]:
_doc_="""
This module defines the utility function in a world where expectations are formed with respect to labour
income uncertainty and survival.
It also defines the functions to calculate expected values and standard deviation with respect to this 
uncertainty.
Assume household will survive with probability psi. 
Household can insure against longevity risk by purchasing annuities.
Household can decide on the fraction to invest in annuities and regular saving assets. 
The investment decision is impacted by the uncertainty of wage in period 2.
Assume that assets have to be greater than a lower threshold a_lower. However, household cannot short-sell
regular assets to purchase annuities.
Assume household receives constant income stream of 1/p_a for annuity and fixed rate R for regular assets.
Assume that wages in period 2 to be log-normally distributed with mean mu_w and variance sig_w.
"""
#Parameters
mu_w = 1.0   #mean of wage
sig_w = 0.3  #variance of wage
n_w = 5      #number of wage nodes
pen = mu_w   #pension that is payed out to the household the last period

R = 1.0                #return on capital
beta = 1.0             #beta is the time discount factor
gamma = 0.5            #intertemporal elasticity of substitution
egam = 1.0 - 1.0/gamma #exponent gamma (1/gamma -> relative risk aversion)
a_lower = 0.0          #lower threshold
psi = np.zeros(3)
psi[1] = 0.8           #probability to survive from period 1 to 2
psi[2] = 0.5           #probability to survive until period 3 conditional on having survived until period 2

#Define Shapes
w = np.zeros(n_w)
weight_w = np.zeros(n_w)
p_a = np.zeros(2)
a = np.zeros(shape=(3,n_w))
omega = np.zeros(shape=(2,n_w))
c = np.zeros(shape=(3,n_w))
wag = np.zeros(shape=(3,n_w))
inc = np.zeros(shape=(3,n_w))
sav = np.zeros(shape=(3,n_w))
alp = np.zeros(shape=(3,n_w))

#utility function of the household
def utility(x):
    """Calculates the negative utility for uncertain lifespan and annuity choice.
    The negative sign allows the utility to be maximized through minimzation. 
    The household is subject to budget constraints in each period (see consumption).
    
    The regular saving asset in period 1 is 0. Saving asset in period 2 is determined by the saving decision 
    in period 1.
    The regular saving asset in period 3 is depends on the uncertain wage from period 2
    ans is, therefore, calculated for each n_w node of wage.
    
    The share of investment in annuities in period 1 and 2 depends on the investment decision
    of the household and is calculated for each n_w node of wage. 
    
    Consumption in period 1 depends on wage and savings decision.
    Consumption in period 2 depends on the return on the regular asset, income from annuity, 
    wage in period 2 and savings in period 2 (i.e. savings asset period 3).
    Consumption in period 3 depends additionally on the pension payed out in period 3. 
    
    Utility is only obtained in periods of survival.
    
    Args:
        x(array of float 64, size ((2*(1+n_w)): saving asset in period 2 and 3 (in period 1 = 0) and
                                                annuity shares in the portfolio
        
    Global Variables:
        a(array of float 64, size (3,n_w): saving asset for period 1-3 at each node
        c(array of float 64, size (3,n_w): consumption for periods 1-3 at each node
        omega(array of float 64 ,size (2, n_w)): share of investment in annuity at each node
    
    Returns:
        array of float64, size (2*(1+n_w)): Utility
    
    """
    global c,a,omega
    #savings
    a[0,:] = 0.0       #saving asset period 1
    a[1,:] = x[0]      #saving asset period 2
    omega[0,:] = x[1]  #share of investment in annuities in period 1
    ic = 2
    for iw in range(n_w): #for loop for each realization of wage
        a[2,iw] = x[ic]       #saving asset period 3
        omega[1,iw] = x[ic+1] #share of investment in annuities in period 1
        ic = ic+2
        
    #consumption (insure consumption >0)
    c[0,:] = mu_w - a[1,0] #consumption in period 1
    c[1,:] = R*(1.0-omega[0,0])*a[1,0] + omega[0,0]*a[1,0]/p_a[0]+w[:] - a[2,:] #consumption in period 2
    c[2,:] = R*(1.0-omega[1,:])*a[2,:] + omega[1,:]*a[2,:]/p_a[1]+omega[0,0]*a[1,0]/p_a[0]+pen #consumption in period 3
    c= np.maximum(c,1e-10) #ensure that consumption is nonnegative to avoid errors
    
    #expected utility of perios 2 and 3
    utility = 0.0
    for iw in range(n_w): #calculates utility at each node
        utility = utility+weight_w[iw]*(c[1,iw]**egam+psi[2]*beta*c[2,iw]**egam)
    
    #add first period utility
    utility = -(c[0,0]**egam+psi[1]*beta*utility)/egam #add utility from period 1
    
    return utility
    
    
#calculates the expected value of x
def E(x):
    """Calculates the expected value when labour income and lifespan  is uncertain"""
    E = 0.0
    for iw in range(n_w):
        E = E + x[iw]*weight_w[iw]
        
    return E

#calculate the standard deviation of x
def Std(x):
    """Calculates the standard deviation when labour income and lifespan  is uncertain"""
    #calculate expected value
    E = 0.0
    for iw in range(n_w):
        E = E + x[iw]*weight_w[iw]
    
    #calculate standard deviation
    Std = 0.0
    for iw in range(n_w):
        Std = Std + x[iw]**2*weight_w[iw]
    Std = np.sqrt(np.maximum(Std-E**2,0.0))
    
    return Std

### 5.5 Program Household 5

In [8]:
"""This program solves the constrained maximization problem for the utility function defined in 5.5m.
The household has to choose the optimal asset level and which fraction of savings to invest in annuities.
The household will receive  constant income stream 1/p_a for all remaining periods. The rest of savings 
will be invested in a regular (bond like) asset with fixed return R.
First the log-normal distribution of wages is discretized generating n_w quadrature nodes.
Then the program minimizes the negative utility function of a household with uncertain 
labour income and lifespan.
The program uses the the function "optimize minimze" from scipy with method SLSQP to find the 
optimal saving asset and annuitiy choice.
We obtain consumption, allocation to the annuity and the regular saving asset.
Finally, we compute the mean and standard deviations of consumption, income and savings for the 3 periods.
"""

#Define shapes
x = np.zeros(2*(1+n_w))
low = np.zeros(2*(1+n_w))
up = np.zeros(2*(1+n_w))

#discretize log(wage)
#Generates n_w quadrature nodes w and weights weight_w for period 2
w = (log_normal_discrete_1(n_w, mu_w, sig_w))[0]        #discretized random variable wage
weight_w = (log_normal_discrete_1(n_w, mu_w, sig_w))[1] #weight for wage

#calculate annuity factors
p_a[0] = psi[1]/R+psi[1]*psi[2]/R**2 #annuity factor in period 1
p_a[1] = psi[2]/R                    #annuity factor in period 2

#lower and upper border and initial guess
low[0] = a_lower #lower bound for savings in period 1
up[0] = mu_w     #upper bound for savings in period 1
low[1] = 0.0     #lower bound for investment in annuities
up[1] = 1.0      #upper bound for investment in annuities
ic = 2
for iw in range(n_w):     #for loop for realizations of n_w
    low[ic] = a_lower     #lower bound saving asset
    up[ic] = R*mu_w+w[iw] #upper bound saving asset
    low[ic+1] = 0.0       #lower bound annuity choice
    up[ic+1] = 1.0        #upper bound annuity choice
    ic = ic+2
    
x=up/2.0 #initial guess

bounds = [(low[i],up[i]) for i in range(2*(1+n_w))] #combine bounds


#minimization routine
result = optimize.minimize(utility,x, #minimize the utility function by changing saving asset and annuity
                  method='SLSQP',     #Define method for minimization
                  bounds=bounds,      #Introduce constraints
                  tol = 1e-14,        #adjust tolerance level for increase in choice variables
                  options={'disp':True})


#set up data for output
for iw in range(n_w):
    wag[0,iw] = mu_w                    #wage in period 1
    inc[0,iw] = mu_w                    #income in period 1
    sav[0,iw] = a[1,0]*(1.0-omega[0,0]) #investment in regular (bond like) asset in period 2 
    alp[0,iw] = a[1,0]*omega[0,0]       #investment in annuity in period 1
    
    wag[1,iw] = w[iw]                     #wage in period 2
    inc[1,iw] = w[iw]+R*(1.0-omega[0,0])*a[1,0]+omega[0,0]*a[1,0]/p_a[0] #income in period 2
    sav[1,iw] = a[2,iw]*(1.0-omega[1,iw]) #investment in regular (bond like) asset in period 2 
    alp[1,iw] = a[2,iw]*omega[1,iw]       #investment in annuity in period 2
    
    wag[2,iw] = 0.0 #wage in period 3
    inc[2,iw] = R*(1.0-omega[1,iw])*a[2,iw]+omega[1,iw]*a[2,iw]/p_a[1]+omega[0,0]*a[1,0]/p_a[0]+pen #income in period 3
    sav[2,iw] = 0.0 #investment in regular (bond like) asset in period 3
    alp[2,iw] = 0.0 #investment in annuity in period 3



#output
#calculate mean and standard deviation
t = PrettyTable(['AGE', 'CONS', 'WAGE', 'INC', 'SREG', 'SANN'])
t.add_row([1,round(E(c[0,:]),2), round(E(wag[0,:]),2), round(E(inc[0,:]),2), round(E(sav[0,:]),2), round(E(alp[0,:]),2)])
t.add_row(['Std' ,round(Std(c[0,:]),2), round(Std(wag[0,:]),2), round(Std(inc[0,:]),2), round(Std(sav[0,:]),2), round(Std(alp[0,:]),2)])
t.add_row([2,round(E(c[1,:]),2), round(E(wag[1,:]),2), round(E(inc[1,:]),2), round(E(sav[1,:]),2), round(E(alp[1,:]),2)])
t.add_row(['Std' ,round(Std(c[1,:]),2), round(Std(wag[1,:]),2), round(Std(inc[1,:]),2), round(Std(sav[1,:]),2), round(Std(alp[1,:]),2)])
t.add_row([3,round(E(c[2,:]),2), round(E(wag[2,:]),2), round(E(inc[2,:]),2), round(E(sav[2,:]),2), round(E(alp[2,:]),2)])
t.add_row(['Std' ,round(Std(c[2,:]),2), round(Std(wag[2,:]),2), round(Std(inc[2,:]),2), round(Std(sav[2,:]),2), round(Std(alp[2,:]),2)])
print(t)


print("E(w) = ",round(sum(weight_w*w),2),"   Var(w) = ",round(sum(weight_w*w**2)-sum(weight_w*w)**2,2))



Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.382416597336106
            Iterations: 40
            Function evaluations: 524
            Gradient evaluations: 40
+-----+------+------+------+------+------+
| AGE | CONS | WAGE | INC  | SREG | SANN |
+-----+------+------+------+------+------+
|  1  | 0.88 | 1.0  | 1.0  | 0.0  | 0.12 |
| Std | 0.0  | 0.0  | 0.0  | 0.0  | 0.0  |
|  2  | 1.03 | 1.0  | 1.1  | 0.0  | 0.07 |
| Std | 0.42 | 0.55 | 0.55 | 0.0  | 0.14 |
|  3  | 1.23 | 0.0  | 1.23 | 0.0  | 0.0  |
| Std | 0.28 | 0.0  | 0.28 | 0.0  | 0.0  |
+-----+------+------+------+------+------+
E(w) =  1.0    Var(w) =  0.3
